## Task 5
Итак... давай, наконец, выясним, повлияло ли посещение ленты новостей на поведение учащихся. 
Приступили ли они в итоге к работе над лабораторным заданием раньше? Помни, что у нас есть две подготовленные 
таблицы в базе данных: `test` и `control`. Мы выполним нечто, похожее на A/B-тестирование.
Чтобы обнаружить эффект, нам нужно вычислить значение дельты (период времени между датой первого коммита и
сроком сдачи лабораторного задания) до того момента, когда учащиеся впервые посетили страницу с лентой новостей, и после этого.
Мы должны сделать то же самое и для контрольной группы.

Другими словами, каждый пользователь из тестовой таблицы имеет свою собственную временную метку для
первого посещения новостной ленты. Мы хотим вычислить среднее значение дельты 
(разницу между датой первого коммита и сроком сдачи) до этой временной метки и после нее. 
Мы сделаем то же самое для пользователей в контрольной группе. Ты можешь сказать: «Но они вообще не посещали ленту новостей». 
Это так, и ранее мы решили использовать среднюю временную метку первого просмотра пользователями тестовой группы для пользователей контрольной группы.

Если значение дельты перед первым посещением ленты новостей значительно отличается от этого показателя после первого 
посещения в тестовой группе, и мы не видим аналогичного эффекта в контрольной группе, значит, создание страницы 
с новостной лентой было отличной идеей. Мы можем распространить эту практику на всю группу.


Подробное описание:

## 1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from pandas import DataFrame

In [2]:
conn = sqlite3.connect("../data/checking-logs.sqlite")

## 2. Используя только **один** запрос для каждой из групп, создай два датафрейма: `test_results` и `control_results` со столбцами `time` и `avg_diff` и только двумя строками.
    - `times` должно иметь значения `before` и `after`.
    - `avg_diff` содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
    - Учитываются только те пользователи, для которых имеются наблюдения и `before`, и `after`.
## 3. Мы по-прежнему не используем лабораторное задание 'project1'.

In [3]:
test_results = pd.io.sql.read_sql("""SELECT  times, AVG(diff)
        FROM
                (SELECT uid, 'before' AS 'times', (diff)
                FROM 
                (SELECT uid, AVG(cast((JULIANDAY(test.first_commit_ts) - 
                JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS integer)) AS diff 
                FROM test 
                LEFT JOIN deadlines AS dl, 
                (SELECT uid AS id, min(first_view_ts) AS min_view
                FROM test
                GROUP BY uid) AS min_view
                ON test.labname = dl.labs AND test.uid = min_view.id 
                WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s') 
                AND test.first_commit_ts < min_view.min_view
                GROUP BY uid)
            UNION
                SELECT uid, 'after' AS 'times', (diff)
                FROM 
                (SELECT uid, AVG(cast((JULIANDAY(test.first_commit_ts) - 
                JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS integer)) AS diff 
                FROM test 
                LEFT JOIN deadlines AS dl, 
                (SELECT uid AS id, min(first_view_ts) AS min_view
                FROM test
                GROUP BY uid) AS min_view
                ON test.labname = dl.labs AND test.uid = min_view.id 
                WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s') 
                AND test.first_commit_ts > min_view.min_view AND 
                test.uid IN 
                (SELECT uid 
                FROM test 
                LEFT JOIN deadlines AS dl, 
                (select uid AS id, min(first_view_ts) AS min_view
                FROM test
                GROUP BY uid) AS min_view
                ON test.labname = dl.labs AND test.uid = min_view.id 
                WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
                AND test.first_commit_ts < min_view.min_view
                GROUP BY uid)
                GROUP BY uid))
        group by times""",
            conn)
test_results

,times,AVG(diff)
0,after,-99.523810
1,before,-66.047619


In [4]:
control_results = pd.io.sql.read_sql("""SELECT  times, AVG(diff)
        FROM
                (SELECT uid, 'before' AS 'times', (diff)
                FROM 
                (SELECT uid, AVG(cast((JULIANDAY(control.first_commit_ts) - 
                JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS integer)) AS diff 
                FROM control 
                LEFT JOIN deadlines AS dl, 
                (SELECT uid AS id, min(first_view_ts) AS min_view
                FROM control
                GROUP BY uid) AS min_view
                ON control.labname = dl.labs AND control.uid = min_view.id 
                WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s') 
                AND control.first_commit_ts < min_view.min_view
                GROUP BY uid)
            UNION
                SELECT uid, 'after' AS 'times', (diff)
                FROM 
                (SELECT uid, AVG(cast((JULIANDAY(control.first_commit_ts) - 
                JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS integer)) AS diff 
                FROM control 
                LEFT JOIN deadlines AS dl, 
                (SELECT uid AS id, min(first_view_ts) AS min_view
                FROM control
                GROUP BY uid) AS min_view
                ON control.labname = dl.labs AND control.uid = min_view.id 
                WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s') 
                AND control.first_commit_ts > min_view.min_view AND 
                control.uid IN 
                (SELECT uid 
                FROM control 
                LEFT JOIN deadlines AS dl, 
                (select uid AS id, min(first_view_ts) AS min_view
                FROM control
                GROUP BY uid) AS min_view
                ON control.labname = dl.labs AND control.uid = min_view.id 
                WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
                AND control.first_commit_ts < min_view.min_view
                GROUP BY uid)
                GROUP BY uid))
        group by times""",
            conn)
control_results

,times,AVG(diff)
0,after,-99.322222
1,before,-98.033333


## 4. Закрой соединение.

In [5]:
conn.close()

## 5. Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?

Да в тестовой группе было различие, а в контрольной — нет.

In [6]:
test_results, control_results

(    times  AVG(diff)
 0   after -99.523810
 1  before -66.047619,
     times  AVG(diff)
 0   after -99.322222
 1  before -98.033333)